#Beam Element - This notebook computes the mass matrix and internal forces and outputs them for later use


1. 2D beam generalized coordinates (1 rotation 2 displacements) (Reduced dimensions so all matricies are subsequently invertible, Mii = 0 if some dof's are accounted for but  unused)
2. Interpolation of roataion matricies WITHOUT orthogonalization (sympy struggling with too much algebra)
3. Circular cross-section with radius 'r'

In [1]:
import numpy as np
import scipy as sp
import sympy as sym
import pickle

from scipy import linalg
from sympy import mpmath
from sympy import cos, sin

from IPython.display import display
from __future__ import division
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')

#### Define Needed Functions

In [2]:
def Skew_sym(v):
    """
    This function returns the skew symetric matrix 
    of the vector 'v' to affect the cross product of 'v'x'u'
    """
    v_matrix = sym.Matrix([[  0 , -v[2]],
                          [v[2],     0]])
    return v_matrix

In [3]:
def Axial_sym(A):
    '''
    This funtcion returns the vector of the skew-symmectric matrix in 2D
    '''
    a_vec = 1/2*sym.Matrix([A[1,0] - A[0,1]])
    return a_vec

In [4]:
def Rotate_sym(theta):
    """
    This function returns the symbolic rotation matrix 
    for the simple 2D-rotation about the third axis
    """
    R = sym.Matrix([[cos(theta),-sin(theta)],
                    [sin(theta), cos(theta)]])
    return R

#### Define symbolic quantites

In [5]:
# symbolic system parameters 
E, I, A, rho, x, l, r = sym.symbols('E I A rho x l r')

# Kinematic values of previos nodes (generic)
# e.g., omega_node  = omega + qdot
theta = sym.Matrix(['theta_1','theta_2'])
omega = sym.Matrix(['omega_1','omega_2'])
alpha = sym.Matrix(['alpha_1','alpha_2'])

# coordinates of the point in the 2D cross-section
# of nodes one and two 
s1 = sym.Matrix(['r_x','r_y'])
s2 = sym.Matrix(['r_x','r_y'])
s = sym.Matrix.vstack(s1,s2)

# generalized coordinates
# one rotation and two displacements per-node (two nodes)
q = sym.Matrix(sym.symarray('q',6))
qdot = sym.Matrix(sym.symarray('qdot',len(q)))
qddot = sym.Matrix(sym.symarray('qddot',len(q)))

# Deformations of Nodes  
u = sym.Matrix([q[1:3,0], q[4:6,0]])
udot = sym.Matrix([qdot[1:3,0], qdot[4:8,0]])
uddot = sym.Matrix([qddot[1:3,0], qddot[4:6,0]])

# display([q,qdot,qddot])
# display([u,udot,uddot])

### Needed Matrix Quantities

In [6]:
# angular velocity for 2D using relative coordinates
# 1D Roation at each node
# Some cheating here q0,q3 and q0dot, q3dot are really 
# the sum of the respective quantites of bodies 1 - bodyj-1 
R1 = Rotate_sym(q[0])
R2 = Rotate_sym(q[3])

omega1_skew = Skew_sym([0,0,qdot[0]])
omega2_skew = Skew_sym([0,0,qdot[3]])

# Only true for planar case
alpha1_skew = Skew_sym([0,0,qddot[0]])
alpha2_skew = Skew_sym([0,0,qddot[3]])

# # 1D Roation at each node
# R1 = Rotate_sym(theta[0])*Rotate_sym(q[0])
# R2 = Rotate_sym(theta[1])*Rotate_sym(q[3])

# omega1_skew = Skew_sym([0,0,omega[0]]) + Skew_sym([0,0,qdot[0]])
# omega2_skew = Skew_sym([0,0,omega[1]]) + Skew_sym([0,0,qdot[3]])

# # Only true for planar case
# alpha1_skew = Skew_sym([0,0,alpha[0]]) + Skew_sym([0,0,qddot[0]])
# alpha2_skew = Skew_sym([0,0,alpha[1]]) + Skew_sym([0,0,qddot[3]])


# display(R1,R2)

# "spatial" rotation matrix
R = sym.Matrix.vstack(sym.Matrix.hstack(R1,sym.zeros(2)), \
                      sym.Matrix.hstack(sym.zeros(2),R2))

# "spatial" angular velocity matrix
Omega_skew = sym.Matrix.vstack(sym.Matrix.hstack(omega1_skew,sym.zeros(2)), \
                               sym.Matrix.hstack(sym.zeros(2),omega2_skew))

# "spatial" angular acceleration matrix
Alpha_skew = sym.Matrix.vstack(sym.Matrix.hstack(alpha1_skew,sym.zeros(2)), \
                               sym.Matrix.hstack(sym.zeros(2),alpha2_skew))
# display(Omega)

### Define Kinematics

In [7]:
# Define velocity of element endpoints (nodes)
v = sym.simplify(udot + R*Omega_skew*s)
print('v = ')
display(v)

# Define acceleration of element endpoints (nodes)
a = sym.simplify(uddot + R*Omega_skew*Omega_skew*s + R*Alpha_skew*s)
print('\na = ')
display(a)

v = 


⎡-q̇₀⋅rₓ⋅sin(q₀) - q̇₀⋅r_y⋅cos(q₀) + q̇₁⎤
⎢                                       ⎥
⎢q̇₀⋅rₓ⋅cos(q₀) - q̇₀⋅r_y⋅sin(q₀) + q̇₂ ⎥
⎢                                       ⎥
⎢-q̇₃⋅rₓ⋅sin(q₃) - q̇₃⋅r_y⋅cos(q₃) + q̇₄⎥
⎢                                       ⎥
⎣q̇₃⋅rₓ⋅cos(q₃) - q̇₃⋅r_y⋅sin(q₃) + q̇₅ ⎦


a = 


⎡                                             2                 2            ⎤
⎢-q̈₀⋅rₓ⋅sin(q₀) - q̈₀⋅r_y⋅cos(q₀) + q̈₁ - q̇₀ ⋅rₓ⋅cos(q₀) + q̇₀ ⋅r_y⋅sin(q₀)⎥
⎢                                                                            ⎥
⎢                                            2                 2             ⎥
⎢q̈₀⋅rₓ⋅cos(q₀) - q̈₀⋅r_y⋅sin(q₀) + q̈₂ - q̇₀ ⋅rₓ⋅sin(q₀) - q̇₀ ⋅r_y⋅cos(q₀) ⎥
⎢                                                                            ⎥
⎢                                             2                 2            ⎥
⎢-q̈₃⋅rₓ⋅sin(q₃) - q̈₃⋅r_y⋅cos(q₃) + q̈₄ - q̇₃ ⋅rₓ⋅cos(q₃) + q̇₃ ⋅r_y⋅sin(q₃)⎥
⎢                                                                            ⎥
⎢                                            2                 2             ⎥
⎣q̈₃⋅rₓ⋅cos(q₃) - q̈₃⋅r_y⋅sin(q₃) + q̈₅ - q̇₃ ⋅rₓ⋅sin(q₃) - q̇₃ ⋅r_y⋅cos(q₃) ⎦

### Compute the Mass Matrix

In [8]:
# Define shape function for element with one node at each end
h = sym.symarray('h', 2)

h[0] = sym.Rational(1,2)*(1 - x)
h[1] = sym.Rational(1,2)*(1 + x)

# Compute shape function matrix
H = sym.expand(sym.Matrix([h[0]*sym.eye(2), h[1]*sym.eye(2)])).T
print('\nH = ')
display(H)

# Define velocity of any point 
Vp = H*v
print('\nV = ')
display(Vp)

# Define velocity of any point 
Ap = H*a
# print('\nA = ')
# display(Accel)

# Compute partial velocities of the nodes
Vr = sym.Matrix([[sym.diff(Vp,qdot) for Vp in Vp] for qdot in qdot]).T
# v_r = H
print('\nVr = ')
display(Vr)
# print(Vr.shape)

# Compute mass matrix
M = sym.factor(sym.Matrix([[sym.expand(sym.integrate(Vr[:,i].dot(Ap)*rho,('r_x',0,r),('r_y',0,r),(x,0,l))).coeff(qddot[j]) \
                            for i in range(len(qddot))] for j in range(len(qddot))]))

pickle.dump( M, open( "gebf-mass-matrix.dump", "wb" ) )


H = 


⎡  x   1           x   1       ⎤
⎢- ─ + ─     0     ─ + ─    0  ⎥
⎢  2   2           2   2       ⎥
⎢                              ⎥
⎢           x   1         x   1⎥
⎢   0     - ─ + ─    0    ─ + ─⎥
⎣           2   2         2   2⎦


V = 


⎡⎛  x   1⎞                                             ⎛x   1⎞                
⎢⎜- ─ + ─⎟⋅(-q̇₀⋅rₓ⋅sin(q₀) - q̇₀⋅r_y⋅cos(q₀) + q̇₁) + ⎜─ + ─⎟⋅(-q̇₃⋅rₓ⋅sin(q₃
⎢⎝  2   2⎠                                             ⎝2   2⎠                
⎢                                                                             
⎢ ⎛  x   1⎞                                            ⎛x   1⎞                
⎢ ⎜- ─ + ─⎟⋅(q̇₀⋅rₓ⋅cos(q₀) - q̇₀⋅r_y⋅sin(q₀) + q̇₂) + ⎜─ + ─⎟⋅(q̇₃⋅rₓ⋅cos(q₃)
⎣ ⎝  2   2⎠                                            ⎝2   2⎠                

                          ⎤
) - q̇₃⋅r_y⋅cos(q₃) + q̇₄)⎥
                          ⎥
                          ⎥
                          ⎥
 - q̇₃⋅r_y⋅sin(q₃) + q̇₅) ⎥
                          ⎦


Vr = 


⎡⎛  x   1⎞                                x   1           ⎛x   1⎞             
⎢⎜- ─ + ─⎟⋅(-rₓ⋅sin(q₀) - r_y⋅cos(q₀))  - ─ + ─     0     ⎜─ + ─⎟⋅(-rₓ⋅sin(q₃)
⎢⎝  2   2⎠                                2   2           ⎝2   2⎠             
⎢                                                                             
⎢⎛  x   1⎞                                         x   1  ⎛x   1⎞             
⎢⎜- ─ + ─⎟⋅(rₓ⋅cos(q₀) - r_y⋅sin(q₀))      0     - ─ + ─  ⎜─ + ─⎟⋅(rₓ⋅cos(q₃) 
⎣⎝  2   2⎠                                         2   2  ⎝2   2⎠             

                 x   1       ⎤
 - r_y⋅cos(q₃))  ─ + ─    0  ⎥
                 2   2       ⎥
                             ⎥
                        x   1⎥
- r_y⋅sin(q₃))     0    ─ + ─⎥
                        2   2⎦

In [9]:
# print('\nM = \n')
# display(M)

print('M_11 = ')
display(M[0:3,0:3])
print('\nM_22 = ')
display(M[3:6,3:6])
print('\nM_12 = ')
display(M[0:3,3:6])
print('\nM_21.T = ')
display(M[3:6,0:3].T)

M_11 = 


⎡   4   ⎛   2          2    ⎞ ⎛ 2          ⎞      3                       ⎛ 2 
⎢l⋅r ⋅ρ⋅⎝sin (q₀) + cos (q₀)⎠⋅⎝l  - 3⋅l + 3⎠  -l⋅r ⋅ρ⋅(sin(q₀) + cos(q₀))⋅⎝l  
⎢───────────────────────────────────────────  ────────────────────────────────
⎢                     18                                           24         
⎢                                                                             
⎢    3                       ⎛ 2          ⎞                 2   ⎛ 2          ⎞
⎢-l⋅r ⋅ρ⋅(sin(q₀) + cos(q₀))⋅⎝l  - 3⋅l + 3⎠              l⋅r ⋅ρ⋅⎝l  - 3⋅l + 3⎠
⎢───────────────────────────────────────────             ─────────────────────
⎢                     24                                           12         
⎢                                                                             
⎢   3                        ⎛ 2          ⎞                                   
⎢l⋅r ⋅ρ⋅(-sin(q₀) + cos(q₀))⋅⎝l  - 3⋅l + 3⎠                                   
⎢──────────────────────────────────────────         


M_22 = 


⎡   4   ⎛   2          2    ⎞ ⎛ 2          ⎞      3                       ⎛ 2 
⎢l⋅r ⋅ρ⋅⎝sin (q₃) + cos (q₃)⎠⋅⎝l  + 3⋅l + 3⎠  -l⋅r ⋅ρ⋅(sin(q₃) + cos(q₃))⋅⎝l  
⎢───────────────────────────────────────────  ────────────────────────────────
⎢                     18                                           24         
⎢                                                                             
⎢    3                       ⎛ 2          ⎞                 2   ⎛ 2          ⎞
⎢-l⋅r ⋅ρ⋅(sin(q₃) + cos(q₃))⋅⎝l  + 3⋅l + 3⎠              l⋅r ⋅ρ⋅⎝l  + 3⋅l + 3⎠
⎢───────────────────────────────────────────             ─────────────────────
⎢                     24                                           12         
⎢                                                                             
⎢   3                        ⎛ 2          ⎞                                   
⎢l⋅r ⋅ρ⋅(-sin(q₃) + cos(q₃))⋅⎝l  + 3⋅l + 3⎠                                   
⎢──────────────────────────────────────────         


M_12 = 


⎡    4   ⎛ 2    ⎞                                          3   ⎛ 2    ⎞       
⎢-l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(sin(q₀)⋅sin(q₃) + cos(q₀)⋅cos(q₃))   l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(sin(q
⎢─────────────────────────────────────────────────────  ──────────────────────
⎢                          18                                            24   
⎢                                                                             
⎢            3   ⎛ 2    ⎞                                            2   ⎛ 2  
⎢         l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(sin(q₃) + cos(q₃))                    -l⋅r ⋅ρ⋅⎝l  -
⎢         ───────────────────────────────────                    ─────────────
⎢                          24                                            12   
⎢                                                                             
⎢           3   ⎛ 2    ⎞                                                      
⎢       -l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(-sin(q₃) + cos(q₃))                                 
⎢       ──────────────────────────────────────      


M_21.T = 


⎡    4   ⎛ 2    ⎞                                          3   ⎛ 2    ⎞       
⎢-l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(sin(q₀)⋅sin(q₃) + cos(q₀)⋅cos(q₃))   l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(sin(q
⎢─────────────────────────────────────────────────────  ──────────────────────
⎢                          18                                            24   
⎢                                                                             
⎢            3   ⎛ 2    ⎞                                            2   ⎛ 2  
⎢         l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(sin(q₃) + cos(q₃))                    -l⋅r ⋅ρ⋅⎝l  -
⎢         ───────────────────────────────────                    ─────────────
⎢                          24                                            12   
⎢                                                                             
⎢           3   ⎛ 2    ⎞                                                      
⎢       -l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(-sin(q₃) + cos(q₃))                                 
⎢       ──────────────────────────────────────      

### Compute Internal forces 

#### 1. Transverse (Bending) Strain

In [10]:
# Orthogonal Matricies Not Extracted to Simplify Algebra
R_interp = sym.simplify(H*sym.Matrix([R1,R2]))
dT = sym.simplify(H.diff(x)*sym.Matrix([R1,R2]))
kappa = sym.simplify(sym.Matrix([Axial_sym(dT*R_interp.T),'0','0','0']))
display(kappa)

⎡-0.5⋅sin(q₀ - q₃)⎤
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎣        0        ⎦

#### 2. Longitudinal (Axial) Strian

In [11]:
# Define Locations of Centroid 
x01 = sym.Matrix(['0','0'])
x02 = sym.Matrix(['1','0'])

# Define Newtonian Unit Vector x-dir
n1 = sym.Matrix(['1','0'])

# Interpolate Displacemnts
u_int = H*u
x_int = H*sym.Matrix([x01,x02])

# Derivatives w.r.t longitudinal beam coordinate
du = u_int.diff(x)
dx0 = x_int.diff(x)

# Compute axial strain
u_ax = dx0 + du - R_interp*n1
epsilon = sym.simplify(sym.Matrix(['0', u_ax[0], '0', u_ax[1]]))
display(epsilon)

⎡                        0                        ⎤
⎢                                                 ⎥
⎢  q₁   q₄   (x - 1)⋅cos(q₀)   (x + 1)⋅cos(q₃)   1⎥
⎢- ── + ── + ─────────────── - ─────────────── + ─⎥
⎢  2    2           2                 2          2⎥
⎢                                                 ⎥
⎢                        0                        ⎥
⎢                                                 ⎥
⎢    q₂   q₅   (x - 1)⋅sin(q₀)   (x + 1)⋅sin(q₃)  ⎥
⎢  - ── + ── + ─────────────── - ───────────────  ⎥
⎣    2    2           2                 2         ⎦

In [12]:
H.diff(x).shape

(2, 4)

#### 3. Compute Internal Forces $Q_e = \frac{\partial U}{\partial e}$

In [13]:
# Transverse strain energy
Ut = 1/2*sym.integrate(E*I*kappa.dot(H.diff(x).T*H.diff(x)*kappa), (x,0,l))
# Longitudinal strain energy
Ul = 1/2*sym.integrate(E*A*epsilon.dot(H.diff(x).T*H.diff(x)*epsilon), (x,0,l))

# Compute Total Energy
U = Ul + Ut

# Compute Internal Force Vector
Qe = sym.Matrix([sym.simplify(sym.expand(sym.diff(-U,q))) for q in q])

####4. Applied and body force vector

In [14]:
# Applied forces
# Gravity body force
fg = -9.81*rho*sym.Matrix([0,1])

# Applied torque
# torque_app = sym.Matrix([0,0,tau_a])

# Compute beta
beta = sym.Matrix([sym.simplify(sym.integrate(Vr[:,j].dot(fg),('r_x',0,r),('r_y',0,r),(x,0,l)))
                   + qe for j,qe in zip(range(len(q)),Qe)])
pickle.dump( beta, open( "gebf-beta.dump", "wb" ) )

In [15]:
display(beta)

⎡    ⎛                           2                                      2     
⎢E⋅l⋅⎜- - -0.0208333333333333⋅A⋅l ⋅sin(q₀ - q₃) + 0.0208333333333333⋅A⋅l ⋅cos(
⎢    ⎝                                                                        
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢      ⎛                      2                     